In [1]:
from functions import *
import numpy as np
import re
import time
import csv

population_number: int = 60 # dimensione della popolazione
iterations_number: int = 35 # numero di generazioni
p_mutation: float = 0.05    # probabilità di mutazione di una soluzione ammissibile

bench_data = []             # matrice tempi di esecuzione job per ogni macchina presi da istanze di Taillard

In [2]:
dictionary = {}
filename = os.getcwd() + '\\instances\\' + '1_taillard_20x5.txt'
file = open(filename, "rt")
with file as f:
    for line in f:
       (key, val) = line.split(':')
       dictionary[key] = val    
#print(dictionary)
file.close()

In [3]:
n_jobs = int(dictionary['number of jobs'])
n_machines = int(dictionary['number of machines'])
best_upper_bound = int(dictionary['upper bound'])
#print(n_jobs)
#print(n_machines)
#print(best_upper_bound)

In [4]:

processing_times_str = re.findall('\[(.*?)\]', dictionary['processing times']) # lista di strnghe che rappresentano le sequenze di job
#print(processing_times)

In [5]:
# trasforma la stringa della sequenza in lista di job
i = 1
for machine_times in processing_times_str:
    list_times = machine_times.split(' ')
    list_times = [int(x) for x in list_times]
    bench_data.append(list_times)
    print('machine' + str(i) + ': ')
    print(list_times)
    i = i + 1
#print('\n')
#print('bench_data: ')
#print(bench_data)

machine1: 
[54, 83, 15, 71, 77, 36, 53, 38, 27, 87, 76, 91, 14, 29, 12, 77, 32, 87, 68, 94]
machine2: 
[79, 3, 11, 99, 56, 70, 99, 60, 5, 56, 3, 61, 73, 75, 47, 14, 21, 86, 5, 77]
machine3: 
[16, 89, 49, 15, 89, 45, 60, 23, 57, 64, 7, 1, 63, 41, 63, 47, 26, 75, 77, 40]
machine4: 
[66, 58, 31, 68, 78, 91, 13, 59, 49, 85, 85, 9, 39, 41, 56, 40, 54, 77, 51, 31]
machine5: 
[58, 56, 20, 85, 53, 35, 53, 41, 69, 13, 86, 72, 8, 49, 47, 87, 58, 18, 68, 28]


In [6]:
processing_time = [] # tempi di esecuzione del job j sulla macchina i
for i in range(n_jobs):
    temp = []
    for j in range(n_machines):
        temp.append(bench_data[j][i])
    print('job' + str(i + 1) + ': ')
    print(temp)
    processing_time.append(temp)


#print(processing_time)

job1: 
[54, 79, 16, 66, 58]
job2: 
[83, 3, 89, 58, 56]
job3: 
[15, 11, 49, 31, 20]
job4: 
[71, 99, 15, 68, 85]
job5: 
[77, 56, 89, 78, 53]
job6: 
[36, 70, 45, 91, 35]
job7: 
[53, 99, 60, 13, 53]
job8: 
[38, 60, 23, 59, 41]
job9: 
[27, 5, 57, 49, 69]
job10: 
[87, 56, 64, 85, 13]
job11: 
[76, 3, 7, 85, 86]
job12: 
[91, 61, 1, 9, 72]
job13: 
[14, 73, 63, 39, 8]
job14: 
[29, 75, 41, 41, 49]
job15: 
[12, 47, 63, 56, 47]
job16: 
[77, 14, 47, 40, 87]
job17: 
[32, 21, 26, 54, 58]
job18: 
[87, 86, 75, 77, 18]
job19: 
[68, 5, 77, 51, 68]
job20: 
[94, 77, 40, 31, 28]


In [7]:
start_time = time.time()
# creazione popolazione
population = initialize_population(population_number, n_jobs)
#print(population)

for evaluation in range(iterations_number):
    print('---- creazione ' + str(evaluation) + ' generazione ----\n')
    parents = get_parents(population, processing_time, n_jobs, n_machines)
    childs = []
    # operatore di crossover per generare nuove sequenze di scheduling
    childs.append(crossover(parents))
   # print('\n')
   # print(childs)

    # operatore di mutazione
    mutated_childs = []
    for child in childs:
        r = np.random.rand()
        if r < p_mutation:
            mutated_child = mutation(child)
            mutated_childs.append(mutated_child)
        mutated_childs.append(child)

    if len(mutated_childs) > 0:
        update_population(population, mutated_childs, processing_time, n_jobs, n_machines)
    print('---- generazione ' + str(evaluation) + ' creata ----\n')


---- creazione 0 generazione ----

fitness cumulata per sequenza: 
(0.01556178026766262, [13, 18, 2, 15, 14, 19, 5, 3, 10, 0, 1, 4, 11, 12, 6, 7, 17, 16, 8, 9])
(0.035480859010270774, [7, 11, 17, 2, 13, 4, 15, 6, 0, 9, 12, 16, 3, 1, 10, 19, 8, 18, 5, 14])
(0.044506691565515094, [6, 11, 19, 14, 9, 15, 18, 16, 17, 13, 3, 10, 8, 12, 0, 2, 1, 7, 5, 4])
(0.0611577964519141, [3, 12, 11, 4, 15, 14, 0, 16, 19, 17, 1, 8, 2, 18, 10, 6, 9, 13, 5, 7])
(0.06924992219109866, [17, 18, 19, 6, 7, 14, 5, 3, 1, 9, 8, 11, 0, 16, 15, 4, 12, 13, 10, 2])
(0.08325552443199502, [9, 1, 7, 6, 17, 13, 19, 16, 18, 15, 4, 12, 8, 3, 11, 10, 0, 5, 14, 2])
(0.09694989106753812, [14, 15, 13, 4, 8, 10, 19, 7, 12, 1, 3, 11, 5, 0, 2, 6, 17, 9, 16, 18])
(0.1151571739807034, [4, 11, 13, 1, 12, 0, 9, 17, 15, 10, 5, 8, 6, 16, 2, 18, 19, 14, 7, 3])
(0.11920323685029567, [17, 19, 1, 2, 16, 15, 11, 10, 0, 3, 5, 18, 12, 6, 9, 8, 7, 4, 14, 13])
(0.13678804855275445, [9, 7, 6, 5, 15, 2, 13, 10, 12, 11, 18, 19, 1, 0, 8, 4, 17, 3, 16

In [8]:
makespans = []
for individual in population:
    ind_makespan = (calc_makespan(individual, processing_time, n_jobs, n_machines), individual)
    makespans.append(ind_makespan)
makespans.sort(key=lambda x: x[0])

best_seq = makespans[0][1]
best_makespan = makespans[0][0]
end_time = time.time()

In [9]:
for ms in makespans:
    print(ms)

(1396, [16, 2, 14, 6, 10, 13, 9, 0, 8, 12, 7, 18, 1, 19, 3, 4, 17, 15, 5, 11])
(1400, [3, 16, 5, 18, 9, 4, 2, 0, 6, 10, 8, 7, 17, 15, 12, 13, 1, 19, 14, 11])
(1414, [5, 4, 10, 12, 11, 18, 8, 7, 0, 14, 2, 3, 13, 19, 9, 1, 16, 15, 17, 6])
(1417, [16, 10, 12, 17, 6, 18, 8, 7, 0, 14, 2, 3, 13, 19, 9, 15, 11, 5, 4, 1])
(1421, [13, 8, 16, 2, 14, 3, 1, 6, 10, 12, 19, 9, 4, 7, 17, 0, 15, 5, 18, 11])
(1426, [5, 2, 3, 12, 0, 4, 10, 18, 8, 11, 17, 7, 13, 15, 16, 9, 6, 19, 14, 1])
(1429, [16, 0, 14, 10, 12, 17, 9, 6, 7, 18, 15, 11, 5, 4, 2, 3, 19, 1, 13, 8])
(1429, [13, 2, 10, 6, 5, 15, 9, 7, 8, 3, 17, 1, 18, 14, 11, 12, 16, 19, 4, 0])
(1430, [1, 8, 2, 14, 7, 9, 18, 13, 5, 4, 16, 15, 0, 17, 11, 12, 19, 3, 10, 6])
(1431, [12, 0, 4, 14, 10, 16, 7, 5, 8, 17, 15, 19, 18, 6, 11, 3, 1, 2, 9, 13])
(1432, [7, 18, 16, 13, 4, 12, 8, 0, 3, 19, 1, 15, 17, 14, 2, 9, 6, 11, 10, 5])
(1434, [1, 13, 4, 12, 14, 17, 8, 11, 10, 5, 16, 6, 15, 7, 3, 0, 19, 2, 18, 9])
(1435, [10, 12, 8, 0, 14, 17, 9, 6, 7, 18, 15, 11, 5

In [10]:
print(best_makespan)

1396


In [11]:
print(best_seq)

[16, 2, 14, 6, 10, 13, 9, 0, 8, 12, 7, 18, 1, 19, 3, 4, 17, 15, 5, 11]


In [12]:
print("--- %s secondi ---" % (end_time - start_time))

--- 0.14416217803955078 secondi ---


In [15]:
dic = [{'instance_dim':filename, 'result': best_makespan, 'execution_time':(end_time - start_time)}]
run_info = ['instance_dim', 'result', 'execution_time']

isfile = os.path.isfile('results.csv')
if not isfile:
    print('non esiste il file')
    with open('results.csv', 'a') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames = run_info)
        writer.writeheader()
        writer.writerows(dic)
else:
    with open('results.csv', 'a') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames = run_info)
        writer.writerows(dic)

non esiste il file
